In [25]:
import sys,os
from py2neo import Graph, Node, Relationship, Database
from py2neo.matching import NodeMatcher
from py2neo.database import Schema

In [26]:
category = ['amazon']  #'dblp', 'youtube']
variant = ['small',] #['medium', 'large'] #Omitted original for now
di = {'amazon':'1', 'dblp':'2', 'youtube':'3', 'small':'4', 'medium':'5', 'large':'6'}

class ID_generators(object):
    def __init__(self, category, variant, dict_identify):
        self.cat = category
        self.var = variant
        self.di = dict_identify
        
    def label_gen(self):
        return self.cat+"_"+self.var
    
    def name_gen(self,i):
        return self.cat+"_"+self.var+"_"+str(i)
    
    def uniq_id(self, i):
        index = str(i)
        index = ''.join(["0" for i in range(0,6-len(index))]) + index
        return int(di[self.cat]+di[self.var]+index)
    
    def gen_cluster_id(self, i, prev=None):
        if prev==None:
            return '|' + str(i) + '|'
        else:
            return prev+str(i)+'|'    

In [28]:
parent_dir = os.environ['GDMPATH']
graph = Graph("bolt:localhost:7474/databases/gdm.db", auth=("neo4j", ""))
graph.delete_all()


In [30]:
graph.delete_all()
schema = Schema(graph)
findNode = NodeMatcher(graph)

for cat in category:
    for var in variant:
        filename = "datasets/{}/{}.graph.{}".format(cat, cat, var)
        Id = ID_generators(cat,var,di)
        tx = graph.begin()
        with open(parent_dir+filename) as fp:
            elements = fp.readline().strip().split(" ")
            for i in range(0, int(elements[0])):
                tx.create(Node(ID=3000, Id.label_gen(), name=Id.name_gen(i),uid=Id.uniq_id(i),cluster=Id.gen_cluster_id(0)))
            tx.commit()
        schema.create_index(ID.label_gen(), 'uid')
        print(filename)    

datasets/amazon/amazon.graph.small


In [46]:
ID = ID_generators('amazon',"small",di)
print(findNode.match(ID.label_gen(), uid=ID.uniq_id(250)).first())

(_2298:amazon_small {cluster: '|0|', name: 'amazon_small_250', uid: 14000250})


In [10]:
for cat in category:
    for var in variant:
        filename = "datasets/{}/{}.graph.{}".format(cat, cat, var)
        ID = ID_generators(cat,var,di)
        tx = graph.begin()
        with open(parent_dir+filename) as fp:
            elements = fp.readline().strip().split(" ")
            for j in range(0, int(elements[1])):
                nodes = fp.readline().strip().split(" ")
                Node1 = findNode.match(ID.label_gen(), uid=ID.uniq_id(int(nodes[0]))).first()
                Node2 = findNode.match(ID.label_gen(), uid=ID.uniq_id(int(nodes[1]))).first()
                tx.create(Relationship(Node1, "KNOWS", Node2))
                tx.create(Relationship(Node2, "KNOWS", Node1))
            tx.commit()
        print(filename)

datasets/amazon/amazon.graph.small
datasets/amazon/amazon.graph.medium
datasets/amazon/amazon.graph.large
datasets/dblp/dblp.graph.small
datasets/dblp/dblp.graph.medium
datasets/dblp/dblp.graph.large
datasets/youtube/youtube.graph.small
datasets/youtube/youtube.graph.medium
datasets/youtube/youtube.graph.large


In [36]:
import time
start = time.time()
for cat in category:
    for var in variant:
        filename = "datasets/{}/{}.graph.{}".format(cat, cat, var)
        ID = ID_generators(cat,var,di)
#         tx = graph.begin()
        with open(parent_dir+filename) as fp:
            elements = fp.readline().strip().split(" ")
            for j in range(0, int(elements[1])):
                nodes = fp.readline().strip().split(" ")
                Node1 = findNode.match(ID.label_gen(), uid=ID.uniq_id(int(nodes[0]))).first()
                Node2 = findNode.match(ID.label_gen(), uid=ID.uniq_id(int(nodes[1]))).first()
end = time.time()
print((end-start)/60)

0.23015296061833698


In [53]:
print(findNode.get(2048))

(_2048:amazon_small {cluster: '|0|', name: 'amazon_small_0', uid: 14000000})


In [59]:
_1142535 = hash(Node(ID.label_gen(), name=ID.name_gen(i),uid=ID.uniq_id(i),cluster=ID.gen_cluster_id(0)))

In [62]:

findNode = NodeMatcher(graph)
findNode.get(139876064017928)

In [68]:
Id = ID_generators('amazon','large',di)
tx = graph.begin()
tx.create(Node(ID=5000, label=Id.label_gen(), name=Id.name_gen(i),uid=Id.uniq_id(i),cluster=Id.gen_cluster_id(0)))
tx.commit()

In [ ]:
tx = graph.begin()
tx.run('CREATE INDEX ON :{}(uid)'.format('amazon_small'))